<a href="https://colab.research.google.com/github/DamnScallion/Visionary-Unicorns-Project/blob/main/yolo/train-IoU-original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone files from our GitHub Repo

In [ ]:
!git clone https://github.com/DamnScallion/Visionary-Unicorns-Project.git

Cloning into 'Visionary-Unicorns-Project'...
remote: Enumerating objects: 11021, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 11021 (delta 43), reused 16 (delta 8), pack-reused 10941
Receiving objects: 100% (11021/11021), 88.17 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Updating files: 100% (10838/10838), done.


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 6.7 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import torch
import torch.nn as nn

In [ ]:
class CIoULoss(nn.Module):
    def __init__(self):
        super(CIoULoss, self).__init__()

    def forward(self, pred, target):
        # Assuming `pred` is the predicted output and `target` is the target tensor with bounding box coordinates
        # Calculate Intersection over Union (IoU)
        iou = torch.sum(torch.min(pred, target)) / torch.sum(torch.max(pred, target))

        # Calculate Complete Intersection over Union (CIoU)
        xA, yA, wA, hA = pred[:, 0], pred[:, 1], pred[:, 2], pred[:, 3]
        xB, yB, wB, hB = target[:, 0], target[:, 1], target[:, 2], target[:, 3]
        C = (wA + wB) * (hA + hB) / (4.0 * (torch.max(xA + wA, xB + wB) - torch.min(xA, xB)) * (torch.max(yA + hA, yB + hB) - torch.min(yA, yB) + 1e-9))

        ciou_loss = 1.0 - iou + C
        return ciou_loss.mean()

In [ ]:
class CustomYOLOv8n(YOLO):
    def __init__(self, config_path):
        super(CustomYOLOv8n, self).__init__(config_path)
        self.loss_fn = CIoULoss()

    def forward(self, x):
        return self.model(x)

In [ ]:
ROOT_DIR = '/content/Visionary-Unicorns-Project/yolo/config'

In [ ]:
model = CustomYOLOv8n('yolov8n.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [ ]:
# Train the model with original given dataset for 10 epochs
results = model.train(data=os.path.join(ROOT_DIR, 'yolo_config.yaml'), epochs=100, imgsz=224, name='yolov8n_ciou_loss_aug')

Ultralytics YOLOv8.0.143 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Visionary-Unicorns-Project/yolo/config/yolo_config_augmented.yaml, epochs=100, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_e10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscr

# Save the model to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!scp -r /content/runs/detect/yolov8n_ciou_loss_aug '/content/drive/MyDrive/COMP9517/YOLO_BCE result'